In [45]:
# %load data_divide - ic.py
import random
import numpy as np
from tqdm import trange


fold=10

dr_pre=np.loadtxt("./source_data/gpcr4_admat_dgc.txt")
simdr=np.loadtxt("./source_data/gpcr_simmat_drug.txt")
simpre=np.loadtxt("./source_data/gpcr_simmat_target.txt")


prenum=len(simpre)

dd_dim1=dr_pre.flatten()

i=0
list_1=[]
while i<len(dd_dim1):
    if dd_dim1[i]==1:
        list_1.append(i)
    i=i+1
num1=len(list_1)
group_size1=int(num1/fold)
random.seed(10)
random.shuffle(list_1)
#
array_1=np.array(list_1)[0:fold*group_size1]
group_data1=np.reshape(array_1,(fold,group_size1))
np.savetxt("./dataset/index_1_gpcr.txt",group_data1)


i=0
list_0=[]
while i<len(dd_dim1):
    if dd_dim1[i]==0:
        list_0.append(i)
    i=i+1
num0=len(list_0)
group_size0=int(num0/fold)
random.seed(10)
random.shuffle(list_0)

array_0=np.array(list_0)[0:fold*group_size0]
group_data0=np.reshape(array_0,(fold,group_size0))
np.savetxt("./dataset/index_0_gpcr.txt",group_data0)


f = 0
for f in trange(fold):
    DTI_gpcr = np.copy(dr_pre)
    i=0
    while i < group_size1:
        r = int(group_data1[f, i] / prenum)
        c = int(group_data1[f, i] % prenum)
        DTI[r, c] = 0
        i += 1  # 得到每次交叉验证中所使用的A矩阵
    np.savetxt("./dataset/DTI_gpcr"+str(f)+".txt",DTI_gpcr)





100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 122.46it/s]


In [71]:
# %load run_main.py
import numpy as np
from function import *
from tqdm import trange
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.metrics
import copy
rs = 0.6
n_fold=10
eta=0.7
l_1=5
l_2=5
fr=500
fp=300
K=20
#dr_dis=np.loadtxt("./source_data/mat_drug_disease.txt")
dr_pre=np.loadtxt("./source_data/gpcr4_admat_dgc.txt").T
#pre_dis=np.loadtxt("./source_data/mat_protein_disease.txt")
#dr_dr=np.loadtxt("./source_data/mat_drug_drug.txt")
#dr_se=np.loadtxt("./source_data/mat_drug_se.txt")
#pre_pre=np.loadtxt("./source_data/mat_protein_protein.txt")
simdr=np.loadtxt("./source_data/gpcr_simmat_drug.txt")
simpre=np.loadtxt("./source_data/gpcr_simmat_target.txt")

drnum=len(simdr)
#disnum=len(dr_dis[0])
prenum=len(simpre)

index_1 = np.loadtxt("./dataset/index_1_gpcr.txt")
index_0 = np.loadtxt("./dataset/index_0_gpcr.txt")
index = np.hstack((index_1, index_0))
reala=dr_pre
sr = simdr
sd = simpre

#RR=np.zeros(dr_pre.shape)
#A = np.hstack((dr_dr, dr_dis, dr_se))
#B = np.hstack((pre_pre, pre_dis))
#cutdr_dis = SVD(A, fr)
#cutpre_dis = SVD(B, fp)
#pre_simdti = cosine_similarity(cutpre_dis, cutpre_dis)
#dr_simdti = cosine_similarity(cutdr_dis, cutdr_dis)
pre_simdti = np.loadtxt("./source_data/S_FGS_p_gpcr.txt")
dr_simdti = np.loadtxt("./source_data/S_FGS_d_gpcr.txt")

srfp, spfp = pruning(K=K, drug_mat=dr_simdti, target_mat=pre_simdti, miu=eta)
srsp, spsp = pruning(K=K, drug_mat=sr, target_mat=sd, miu=eta)

for f in trange(n_fold):
    a = np.loadtxt("./dataset/DTI_gpcr" + str(f) + ".txt").T
    idx=index[f,:]
   
    R = copy.deepcopy(a)
    R = BiRW(R, dr_simdti, pre_simdti, a, rs)        
    for i in trange(l_1):
        R=BiRW(R, srfp, spfp, a, rs/2)
    for i in trange(l_2):
        R=BiRW(R, srsp, spsp, a, rs/2)

    realvalue=np.zeros(R.shape)
    for i in range(len(idx)):
        d=int(idx[i]/prenum)
        p=int(idx[i]%prenum)
    
        realvalue[d,p]=reala[d,p]
    for i in range(len(idx)):
        d=int(idx[i]/prenum)
        p=int(idx[i]%prenum)
        RR[d,p]=R[d,p]

y_true_m = dr_pre.tolist()
y_pre_m = RR.tolist()

tpr_cov=[[] for i in range(n_fold)]
fpr_cov=[[] for i in range(n_fold)]
recall_cov=[[] for i in range(n_fold)]
precision_cov=[[] for i in range(n_fold)]

for f in range(n_fold):
    idx=index[f,:]
    singal=[[] for i in range(len(y_true_m))]
    singal_test=[[] for i in range(len(y_true_m))]
    for i in trange(len(y_true_m)):
        for j in range(len(y_true_m[0])):
            if i*prenum+j in idx:
                singal[i].append(y_true_m[i][j])
                singal_test[i].append(y_pre_m[i][j])
    y_true=singal
    y_pre=singal_test

    idx = []
    tpr_list = []
    fpr_list = []
    recall_list = []
    precision_list = []
    c = 0
    for i in trange(len(y_true)):
        if np.sum(np.array(y_true[i])) == 0:
            c += 1
            continue
        else:
            tpr1, fpr1, precision1, recall1 = tpr_fpr_precision_recall(np.array(y_true[i]), np.array(y_pre[i]))
            fpr_list.append(fpr1)
            tpr_list.append(tpr1)
            precision_list.append(precision1)
            recall_list.append(recall1)
    coverage = []

    for i in tpr_list:
        try:
            coverage.append(i.index(1.0)+1)
        except:
            print('1')
    print(np.mean(np.array(coverage)))

    tpr = equal_len_list(tpr_list)
    print(len(tpr ),type(tpr))
    fpr = equal_len_list(fpr_list)
    precision = equal_len_list(precision_list)
    recall = equal_len_list(recall_list)
    tpr_mean = np.mean(tpr, axis=0)
    tpr_cov[f]=tpr_mean
    print(type(tpr_mean),len(tpr_mean))
    fpr_mean = np.mean(fpr, axis=0)
    fpr_cov[f]=fpr_mean
    recall_mean = np.mean(recall, axis=0)
    recall_cov[f]=recall_mean
    precision_mean = np.mean(precision, axis=0)
    precision_cov[f]=precision_mean
    print('The auc of prediction is:', sklearn.metrics.auc(fpr_mean, tpr_mean))
    print('The aupr of prediction is:', sklearn.metrics.auc(recall_mean, precision_mean)+recall_mean[0]*precision_mean[0])

tpr = equal_len_list(tpr_cov)
fpr = equal_len_list(fpr_cov)
precision = equal_len_list(precision_cov)
recall = equal_len_list(recall_cov)

tpr_mean = np.mean(tpr, axis=0)
fpr_mean = np.mean(fpr, axis=0)
recall_mean = np.mean(recall, axis=0)
precision_mean = np.mean(precision, axis=0)

print('The auc of prediction is:', sklearn.metrics.auc(fpr_mean, tpr_mean))
print('The aupr of prediction is:', sklearn.metrics.auc(recall_mean, precision_mean)+recall_mean[0]*precision_mean[0])

np.savetxt('./result/fpr_list.txt', fpr_mean)
np.savetxt('./result/tpr_list.txt', tpr_mean)
np.savetxt('./result/recall_list.txt', recall_mean)
np.savetxt('./result/precision_list.txt', precision_mean)

In [47]:
dr_pre=np.loadtxt("./source_data/gpcr4_admat_dgc.txt")
#pre_dis=np.loadtxt("./source_data/mat_protein_disease.txt")
#dr_dr=np.loadtxt("./source_data/mat_drug_drug.txt")
#dr_se=np.loadtxt("./source_data/mat_drug_se.txt")
#pre_pre=np.loadtxt("./source_data/mat_protein_protein.txt")
simdr=np.loadtxt("./source_data/gpcr_simmat_drug.txt")
simpre=np.loadtxt("./source_data/gpcr_simmat_target.txt")

In [41]:
simdr.shape

(223, 223)

In [40]:
idx

array([11470.,   771.,  4649., ...,  9990., 13051.,  7740.])

In [43]:
reala.shape

(95, 223)

In [44]:
p,d

(70, 120)

In [53]:
a.shape

(223, 95)

In [56]:
realvalue[d,p]

0.0

In [57]:
reala

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

10

In [62]:
sd

array([[0.01      , 0.00282628, 0.00083495, ..., 0.00019751, 0.00068153,
        0.00199133],
       [0.00282628, 0.01      , 0.0006315 , ..., 0.00022949, 0.00051822,
        0.00203509],
       [0.00083495, 0.0006315 , 0.01      , ..., 0.0001932 , 0.0005582 ,
        0.00048957],
       ...,
       [0.00019751, 0.00022949, 0.0001932 , ..., 0.01      , 0.00023548,
        0.00029656],
       [0.00068153, 0.00051822, 0.0005582 , ..., 0.00023548, 0.01      ,
        0.00051825],
       [0.00199133, 0.00203509, 0.00048957, ..., 0.00029656, 0.00051825,
        0.01      ]])

In [63]:
simpre

array([[1.        , 0.28262829, 0.08349522, ..., 0.01975119, 0.06815311,
        0.19913333],
       [0.28262829, 1.        , 0.06314987, ..., 0.02294885, 0.05182183,
        0.2035093 ],
       [0.08349522, 0.06314987, 1.        , ..., 0.01932024, 0.05582016,
        0.0489569 ],
       ...,
       [0.01975119, 0.02294885, 0.01932024, ..., 1.        , 0.02354822,
        0.0296562 ],
       [0.06815311, 0.05182183, 0.05582016, ..., 0.02354822, 1.        ,
        0.05182548],
       [0.19913333, 0.2035093 , 0.0489569 , ..., 0.0296562 , 0.05182548,
        1.        ]])